## Doing Augmentation

https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

In [1]:
import os
import albumentations as alb
import cv2
import matplotlib.pyplot as plt

## Working perfect

In [88]:
augmentor_without_boudingbox = alb.Compose([alb.RandomCrop(width=450, height=450), 
            alb.HorizontalFlip(p=0.5), 
            alb.RandomBrightnessContrast(p=0.5),
            alb.RandomGamma(p=0.5), 
            alb.RGBShift(p=0.5), 
            alb.VerticalFlip(p=0.5)])
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                        alb.HorizontalFlip(p=0.5), 
                        alb.RandomBrightnessContrast(p=0.5),
                        alb.RandomGamma(p=0.5), 
                        alb.RGBShift(p=0.5), 
                        alb.VerticalFlip(p=0.5)], 
                    bbox_params=alb.BboxParams(format='yolo', 
                                                label_fields=['class_labels']))

In [87]:
for partition in ['test']: 
    for image in os.listdir(os.path.join(partition, 'images')):
        img = cv2.imread(os.path.join(partition, 'images', image))

        coords = [0,0,0,0]
        label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.readline().strip("\n")
            label = [float(value) for value in content.split(" ")]
            coords[0] = label[1]
            coords[1] = label[2]
            coords[2] = label[3]
            coords[3] = label[4]
        for x in range(2):
            if coords != [0,0,0,0] or coords != [0.0,0.0,0.0,0.0]:
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
            else:
                augmented = augmentor_without_boudingbox(image=img)
            directory_path = os.path.join('aug_data', partition, 'images')
            os.makedirs(directory_path, exist_ok=True)
            cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

            annotation = [0,0,0,0,0]

            if os.path.exists(label_path):
                if len(augmented['bboxes']) == 0: 
                    annotation[1] = 0
                    annotation[2] = 0
                    annotation[3] = 0
                    annotation[4] = 0
                    annotation[0] = 0 
                else: 
                    annotation[1] = augmented['bboxes'][0][0]
                    annotation[2] = augmented['bboxes'][0][1]
                    annotation[3] = augmented['bboxes'][0][2]
                    annotation[4] = augmented['bboxes'][0][3]
                    annotation[0] = 1
            else: 
                    annotation[1] = 0
                    annotation[2] = 0
                    annotation[3] = 0
                    annotation[4] = 0
                    annotation[0] = 0 
            directory_path_labels = os.path.join('aug_data', partition, 'labels')
            os.makedirs(directory_path_labels,exist_ok=True)
            with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'), 'w') as f:
                f.write(' '.join(map(str, annotation)))


In [ ]:
loc_img = "dummy/test/images/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.jpg"
loc_annot = "dummy/test/labels/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.txt"
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread(loc_img)
with open(loc_annot,'r') as f:
    value = f.readline().strip()
    value_list = value.split()
value_list = [float(val)for val in value_list]

coords = value_list[1:]

augmented = augmentor(image=img,bboxes=[coords],class_labels=['face'])

# Assuming augmented['image'] is the augmented image you want to draw on
augmented_img =augmented['image']

# Assuming augmented['bboxes'][0] contains the augmented bounding box coordinates
augmented_bbox = augmented['bboxes'][0]

# Original image size
original_width = 640
original_height = 480

# Augmented image size
augmented_width = 450
augmented_height = 450

# Calculate the scale factors for width and height
width_scale = augmented_width / original_width
height_scale = augmented_height / original_height

# Scale the augmented bounding box coordinates to match the augmented image size
x_center = augmented_bbox[0] * original_width  # x-coordinate of the box center
y_center = augmented_bbox[1] * original_height  # y-coordinate of the box center
box_width = augmented_bbox[2] * original_width  # width of the box
box_height = augmented_bbox[3] * original_height  # height of the box

# Calculate the top-left and bottom-right coordinates of the box
x_min = int((x_center - (box_width / 2)) * width_scale)
y_min = int((y_center - (box_height / 2)) * height_scale)
x_max = int((x_center + (box_width / 2)) * width_scale)
y_max = int((y_center + (box_height / 2)) * height_scale)

new_coords = [x_min,y_min,x_max,y_max]

# Draw the rectangle on the augmented image
color = (255, 0, 0)  # Blue color
thickness = 2  # Thickness of the rectangle border
cv2.rectangle(augmented_img, (x_min, y_min), (x_max, y_max), color, thickness)

# Display the augmented image with the rectangle
plt.imshow(augmented_img)
plt.show()

## Trying out

In [89]:
img = cv2.imread("9e9e02a0-f7fe-11ed-ab83-38d547b2c17a.jpg")

coords = [0,0,0,0]
label_path = "9e9e02a0-f7fe-11ed-ab83-38d547b2c17a.txt"
if os.path.exists(label_path):
    with open(label_path, 'r') as f:
        content = f.readline().strip("\n")
    label = [float(value) for value in content.split(" ")]
    coords[0] = label[1]
    coords[1] = label[2]
    coords[2] = label[3]
    coords[3] = label[4]
for x in range(2):
    if coords != [0,0,0,0] or coords != [0.0,0.0,0.0,0.0]:
        augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
    else:
        augmented = augmentor_without_boudingbox(image=img)
    directory_path = os.path.join('dummy', partition, 'images')
    os.makedirs(directory_path, exist_ok=True)
    cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

    annotation = [0,0,0,0,0]

    if os.path.exists(label_path):
        if len(augmented['bboxes']) == 0: 
            annotation[1] = 0
            annotation[2] = 0
            annotation[3] = 0
            annotation[4] = 0
            annotation[0] = 0 
        else: 
            annotation[1] = augmented['bboxes'][0][0]
            annotation[2] = augmented['bboxes'][0][1]
            annotation[3] = augmented['bboxes'][0][2]
            annotation[4] = augmented['bboxes'][0][3]
            annotation[0] = 1
    else: 
            annotation[1] = 0
            annotation[2] = 0
            annotation[3] = 0
            annotation[4] = 0
            annotation[0] = 0 
    directory_path_labels = os.path.join('dummy', partition, 'labels')
    os.makedirs(directory_path_labels,exist_ok=True)
    with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'), 'w') as f:
        f.write(' '.join(map(str, annotation)))

In [98]:
img = cv2.imread("Garden3.jpg")

coords_list = []
label_path = "Garden3.txt"
if os.path.exists(label_path):
    with open(label_path, 'r') as f:
        content = f.readlines()

    for line in content:
        label = [float(value) for value in line.strip().split(" ")]
        coords = label[1:5]
        coords_list.append(coords)

for x in range(2):
    if coords_list:
        augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
    else:
        augmented = augmentor_without_boudingbox(image=img)

    directory_path = os.path.join('dummy', partition, 'images')
    os.makedirs(directory_path, exist_ok=True)
    cv2.imwrite(
        os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),
        augmented['image']
    )

    annotation = []

    if len(augmented['bboxes']) == 0:
        annotation.append(0)
    else:
        for bbox in augmented['bboxes']:
            annotation.append(1)
            annotation.extend(bbox)

    directory_path_labels = os.path.join('dummy', partition, 'labels')
    os.makedirs(directory_path_labels, exist_ok=True)
    with open(
        os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),
        'w'
    ) as f:
        f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))

In [ ]:
loc_img = "dummy/test/images/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.jpg"
loc_annot = "dummy/test/labels/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.txt"
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread(loc_img)
with open(loc_annot,'r') as f:
    value = f.readline().strip()
    value_list = value.split()
value_list = [float(val)for val in value_list]

coords = value_list[1:]

augmented = augmentor(image=img,bboxes=[coords],class_labels=['face'])

# Assuming augmented['image'] is the augmented image you want to draw on
augmented_img =augmented['image']

# Assuming augmented['bboxes'][0] contains the augmented bounding box coordinates
augmented_bbox = augmented['bboxes'][0]

# Original image size
original_width = 640
original_height = 480

# Augmented image size
augmented_width = 450
augmented_height = 450

# Calculate the scale factors for width and height
width_scale = augmented_width / original_width
height_scale = augmented_height / original_height

# Scale the augmented bounding box coordinates to match the augmented image size
x_center = augmented_bbox[0] * original_width  # x-coordinate of the box center
y_center = augmented_bbox[1] * original_height  # y-coordinate of the box center
box_width = augmented_bbox[2] * original_width  # width of the box
box_height = augmented_bbox[3] * original_height  # height of the box

# Calculate the top-left and bottom-right coordinates of the box
x_min = int((x_center - (box_width / 2)) * width_scale)
y_min = int((y_center - (box_height / 2)) * height_scale)
x_max = int((x_center + (box_width / 2)) * width_scale)
y_max = int((y_center + (box_height / 2)) * height_scale)

new_coords = [x_min,y_min,x_max,y_max]

# Draw the rectangle on the augmented image
color = (255, 0, 0)  # Blue color
thickness = 2  # Thickness of the rectangle border
cv2.rectangle(augmented_img, (x_min, y_min), (x_max, y_max), color, thickness)

# Display the augmented image with the rectangle
plt.imshow(augmented_img)
plt.show()

## making suitable (Finalized Version)

In [3]:
import cv2
import albumentations as alb
import os

In [12]:
W = 1280
H = 720

In [13]:
augmentor_without_boudingbox = alb.Compose([alb.RandomCrop(width=W, height=H), 
            alb.HorizontalFlip(p=0.5), 
            alb.RandomBrightnessContrast(p=0.5),
            alb.Rotate(limit=30, p=0.5),
            alb.Blur(p=0.2),
            alb.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.2),
            alb.RandomGamma(p=0.5), 
            alb.RGBShift(p=0.5), 
            alb.VerticalFlip(p=0.5)])
augmentor = alb.Compose([alb.RandomCrop(width=W, height=H), 
                        alb.HorizontalFlip(p=0.5), 
                        alb.RandomBrightnessContrast(p=0.5),
                        alb.Rotate(limit=30, p=0.5),
                        alb.Blur(p=0.2),
                        alb.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.2),
                        alb.RandomGamma(p=0.5), 
                        alb.RGBShift(p=0.5), 
                        alb.VerticalFlip(p=0.5)], 
                    bbox_params=alb.BboxParams(format='yolo', 
                                                label_fields=['class_labels']))

In [4]:
for partition in ['Train(2)']: 
    for image in os.listdir(os.path.join(partition, 'images')):
        img = cv2.imread(os.path.join(partition, 'images', image))
        coords_list = []
        class_value = []
        label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.readlines()
            for line in content:
                label = [float(value) for value in line.strip().split(" ")]
                class_value.append(int(label[0]))
                coords = label[1:5]
                coords_list.append(coords)
        for x in range(2):
            if coords_list:
                augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
            else:
                augmented = augmentor_without_boudingbox(image=img)
            directory_path = os.path.join('dummy', partition, 'images')
            os.makedirs(directory_path, exist_ok=True)
            cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
            annotation = []
            if os.path.exists(label_path):
                for i in range(len(coords_list)):
                    if i < len(augmented['bboxes']):
                        annotation.append(class_value[i])
                        annotation.extend(augmented['bboxes'][i])
                    else:
                        annotation.append(0)
                        annotation.extend([0, 0, 0, 0])
            else:
                annotation.append(0)
                annotation.append(0)
                annotation.append(0)
                annotation.append(0)
                annotation.append(0)
       
            directory_path_labels = os.path.join('dummy', partition, 'labels')
            os.makedirs(directory_path_labels, exist_ok=True)
            with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),'w') as f:
                f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))

In [13]:
org_image = "Garden4.jpg"
org_label = "Garden4.txt"

image_path = "Dummy\Train\images\Garden1.2.jpg"
label_path = "Dummy\Train\labels\Garden1.2.txt"

image = cv2.imread(image_path)
# Resize image for display
scale_percent = 50  # Adjust the value as needed
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
image = cv2.resize(image, (width, height))

with open(label_path, 'r') as f:
    lines = f.readlines()

for line in lines:
    label = line.strip().split(" ")
    class_id = int(label[0])
    x_center = float(label[1]) * image.shape[1]
    y_center = float(label[2]) * image.shape[0]
    width = float(label[3]) * image.shape[1]
    height = float(label[4]) * image.shape[0]

    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    # Draw bounding box rectangle
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Display class label
    class_label = f"{class_id}"
    cv2.putText(image, class_label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the image with bounding box annotations
cv2.imshow("Image with Bounding Boxes", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Rename Files do our desired names

In [ ]:
import os
folder_images_path = ["Train\images"]
folder_label_path = ["Train\label"]
Give_Me_Name = "New"

for dir in folder_images_path:
    file_list = os.listdir(dir)

    for i, filename in enumerate(file_list):
        new_filename = f"{Give_Me_Name}{i+1}.jpg"
        old_filepath = os.path.join(dir, filename)
        new_filepath = os.path.join(dir, new_filename)
        os.rename(old_filepath, new_filepath)
        
        print(f"Renamed {filename} to {new_filename}")
for dir in folder_label_path:
    file_list = os.listdir(dir)

    for i,filename in enumerate(file_list):
        new_filename = f"{Give_Me_Name}{i+1}.txt"
        old_filepath = os.path.join(dir,filename)
        new_filepath = os.path.join(dir,new_filename)
        os.rename(old_filepath,new_filepath)

        print(f"Renamed {filename} to {new_filename}")